# Task Settings

In [1]:
import os
os.environ['OPENAI_API_KEY']=""


from utils import *
from prompts import Prompts, TASK_LABLES, TAGS

dataset_name = 'trafficsafe'                   # 'trafficsafe', 'swiss'
test_model = "experimental:gpt-4o-mini"     # Forward Engine
eval_model = "gpt-4o"                       # Backward Engine
iteration = 1
date = '0919'
total_steps=9                               # 5-11
epoch=1                                     # 2, 1
batch_size=3                                # 1-3

# Initialize

In [2]:
cm_labels = TASK_LABLES[dataset_name]
tags = TAGS[dataset_name]
CAUSAL_SYSTEM = Prompts[dataset_name]['CAUSAL_SYSTEM']
CAUSAL_SYSTEM_CONSTRAINT = Prompts[dataset_name]['CAUSAL_SYSTEM_CONSTRAINT']
SYSTEM = Prompts[dataset_name]['SYSTEM']

llm_api_eval = tg.get_engine(engine_name=eval_model)
llm_api_test = tg.get_engine(engine_name=test_model, cache=False)
tg.set_backward_engine(llm_api_eval, override=True)

train_set, val_set, test_set_ori, eval_fn = load_task(dataset_name, evaluation_api=llm_api_eval, prompt_col="organized_prompt")
train_loader = tg.tasks.DataLoader(train_set, batch_size=batch_size, shuffle=True)
col = "organized_prompt" if dataset_name == 'swiss' else "prompt"
train_set.data[col] = train_set.data[col].apply(lambda x: f"{tags[0]}{x}{tags[1]}")
val_set.data[col] = val_set.data[col].apply(lambda x: f"{tags[0]}{x}{tags[1]}")
test_set_ori.data[col] = test_set_ori.data[col].apply(lambda x: f"{tags[0]}{x}{tags[1]}")
print("Train/Val/Test Set Lengths: ", len(train_set), len(val_set), len(test_set_ori))

Train/Val/Test Set Lengths:  100 100 100


In [3]:
import matplotlib.pyplot as plt
from copy import deepcopy

res = []

test_set = deepcopy(test_set_ori)
system_prompt, causal_prompt, model, causal_model, optimizer, optimizer_causal = init(SYSTEM, CAUSAL_SYSTEM, llm_api_test, llm_api_eval, CAUSAL_SYSTEM_CONSTRAINT)
results = {"test_f1": [], "prompt": [], "validation_f1": [], 'system_prompt':[], 'causal_prompt': []}

In [4]:
acc, t_rs, t_ys = eval_dataset(test_set, eval_fn, model,iters=1)
test_res = evaluate_metrics(t_rs, t_ys, cm_labels)

print(test_res)

Accuracy: 0.2700: 100%|██████████| 100/100 [00:21<00:00,  4.64it/s]

{'accuracy': 0.27, 'precision': 0.18028571428571427, 'f1': 0.19697097647836564, 'cm': array([[ 2, 19,  2,  0],
       [ 4, 16,  4,  0],
       [ 0, 13,  9,  0],
       [ 4, 15, 12,  0]])}


In [5]:
acc, t_rs, t_ys = eval_dataset(test_set, eval_fn, model,iters=1)
test_res = evaluate_metrics(t_rs, t_ys, cm_labels)

print(test_res)

Accuracy: 0.3500: 100%|██████████| 100/100 [00:11<00:00,  8.85it/s]

{'accuracy': 0.35, 'precision': 0.2624962779156328, 'f1': 0.2682188160676533, 'cm': array([[ 4, 18,  0,  0],
       [ 0, 20,  4,  0],
       [ 2,  8, 11,  0],
       [ 4, 16, 11,  0]])}


In [6]:
acc, t_rs, t_ys = eval_dataset(test_set, eval_fn, model,iters=1)
test_res = evaluate_metrics(t_rs, t_ys, cm_labels)

print(test_res)

Accuracy: 0.3000: 100%|██████████| 100/100 [00:18<00:00,  5.38it/s]

{'accuracy': 0.3, 'precision': 0.19343406593406595, 'f1': 0.21563468265867067, 'cm': array([[ 3, 16,  4,  0],
       [ 2, 20,  2,  0],
       [ 1, 14,  7,  0],
       [ 7, 13, 11,  0]])}


# Run EGO-Prompt

In [4]:
import time
import copy
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

def run_one_worker(worker_id: int):
    
    local_test_set = copy.deepcopy(test_set)
    local_test_set_ori = copy.deepcopy(test_set_ori)
    local_val_set = copy.deepcopy(val_set)
    local_train_loader = train_loader

    val_performance = -float('inf')
    test_performance = -float('inf')
    final_results = None
    all_val_f1s = []
    all_test_f1s = []

    local_test_set.data[col] = local_test_set_ori.data[col].apply(
        lambda x: f"<!-- {time.time()} (w{worker_id}) -->, {x}"
    )

    for cur_iter in range(iteration):
        print(f"[Worker {worker_id}] [Iteration {cur_iter+1}/{iteration}] begin")
        output_json = (
            f"res/{date}_{dataset_name}_{test_model.split('/')[-1].split(':')[-1]}_"
            f"w{worker_id}_it{cur_iter+1}.json"
        )
        initialize_json_file(output_json)

        system_prompt, causal_prompt, model, causal_model, optimizer, optimizer_causal = init(
            SYSTEM, CAUSAL_SYSTEM, llm_api_test, llm_api_eval, CAUSAL_SYSTEM_CONSTRAINT
        )

        results, test_res, val_res = init_eval(
            local_val_set, local_test_set, eval_fn, model, causal_model,
            system_prompt, causal_prompt, cm_labels, iters=ITERS    
        )

        results = run_training(
            local_train_loader, local_val_set, local_test_set, eval_fn,
            model, causal_model, system_prompt, causal_prompt,
            optimizer, optimizer_causal, results, cm_labels,
            output_json=output_json, epoch=epoch, steps=total_steps, iters=ITERS
        )

        all_val_f1s.append(results['validation_f1'])
        all_test_f1s.append(results['test_f1'])

        cur_val = results['validation_f1'][-1]
        cur_test = results['test_f1'][-1]
        if cur_val > val_performance:
            val_performance = cur_val
            test_performance = cur_test
            final_results = results

        print(f"[Worker {worker_id}] [Iteration {cur_iter+1}] "
              f"val_best={val_performance:.4f}, test_at_best={test_performance:.4f}")

    return {
        'best_test_f1': test_performance,
        'val_f1s': all_val_f1s,
        'test_f1s': all_test_f1s,
        'worker_id': worker_id,
    }


# Running the cell below will incur API usage charges. Refer to our paper for detailed cost

In [5]:
NUM_WORKERS = 3
ITERS = 1
total_steps=9
epoch=1
EGO_res = []


with ThreadPoolExecutor(max_workers=NUM_WORKERS) as ex:
    futures = [ex.submit(run_one_worker, i) for i in range(NUM_WORKERS)]
    for fut in as_completed(futures):
        res = fut.result()

        EGO_res.append(res['best_test_f1'])

        print(f"[Main] Worker {res['worker_id']} done. Best test_f1={res['best_test_f1']:.4f}")

print("EGO_res (best test F1 per worker):", EGO_res)

[Worker 0] [Iteration 1/1] begin
[Worker 2] [Iteration 1/1] begin
[Worker 1] [Iteration 1/1] begin


Accuracy: 0.2600: 100%|██████████| 100/100 [00:20<00:00,  4.83it/s]


SCG_val_f1: 0.1821413043478261, SCG_test_f1:0.21471328825114078


Accuracy: 0.2987:  75%|███████▌  | 75/100 [00:15<00:02,  9.26it/s]


Epoch 0, Step 0


...

[Test Result] F1: 0.3126
[Worker 2] [Iteration 1] val_best=0.3640, test_at_best=0.3126
[Main] Worker 2 done. Best test_f1=0.3126


Accuracy: 0.3500: 100%|██████████| 100/100 [00:20<00:00,  4.93it/s]


Skip Test
[System Validation] F1: 0.3059, Previous F1: 0.3224
[System Validation CM]:
[[ 4  8 10  0]
 [ 2 12  8  0]
 [ 1  7 16  0]
 [ 2  8 18  3]]


Accuracy: 0.2400: 100%|██████████| 100/100 [00:21<00:00,  4.72it/s]
9it [40:22, 269.14s/it]

Skip Test
[Causal Validation] F1: 0.2056, Previous F1: 0.3224
[Causal Validation CM]:
[[ 1  8 14  0]
 [ 3  4 15  0]
 [ 1  7 15  1]
 [ 4  7 16  4]]
Skip Test
[Worker 1] [Iteration 1] val_best=0.3224, test_at_best=0.3051
[Main] Worker 1 done. Best test_f1=0.3051
EGO_res (best test F1 per worker): [0.3291249143880723, 0.312577579697298, 0.30512086461513077]
